# CS4225/CS5425 Assignment 2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# For Google Colaboratory
!pip install pyspark py4j
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/MyDrive/CS4225/Assignment_2/'
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/MyDrive/CS4225/Assignment_2


## 1. Data Preparation

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

load dataset

In [ ]:
transactions_path = 'file:/content/gdrive/MyDrive/CS4225/Assignment_2/transactions/transactions_*.csv'
transaction_items_path = 'file:/content/gdrive/MyDrive/CS4225/Assignment_2/transaction_items/transaction_items_*.csv'
users_path = 'file:/content/gdrive/MyDrive/CS4225/Assignment_2/users/users_*.csv'
stores_path = 'file:/content/gdrive/MyDrive/CS4225/Assignment_2/stores/stores.csv'
menu_path = 'file:/content/gdrive/MyDrive/CS4225/Assignment_2/menu_items/menu_items.csv'

transactions = spark.read.csv(transactions_path, header=True, inferSchema=True)
transaction_items = spark.read.csv(transaction_items_path, header=True, inferSchema=True)
users = spark.read.csv(users_path, header=True, inferSchema=True)
stores = spark.read.csv(stores_path, header=True, inferSchema=True)
menu = spark.read.csv(menu_path, header=True, inferSchema=True)

date

In [ ]:
transactions = transactions.withColumn("transaction_date", to_date(col("created_at")))
transaction_items = transaction_items.withColumn("transaction_date", to_date(col("created_at")))

transactions = transactions.na.drop(subset=["transaction_date", "original_amount", "store_id"])
transaction_items = transaction_items.na.drop(subset=["transaction_date", "subtotal", "item_id", "quantity"])

store_sales_T

In [ ]:
daily_sales = transactions.groupBy("store_id", "transaction_date").agg(
    sum("original_amount").alias("store_sales_T")
)
daily_sales.show(5)
daily_sales.count()

+--------+----------------+-------------+
|store_id|transaction_date|store_sales_T|
+--------+----------------+-------------+
|      10|      2025-05-27|      67032.5|
|       2|      2025-03-16|      65408.5|
|       7|      2025-05-07|      65853.5|
|       5|      2025-05-16|      68606.0|
|       1|      2025-03-30|      64701.5|
+--------+----------------+-------------+
only showing top 5 rows



7310

store_sales_T-1

In [ ]:
from pyspark.sql.window import Window

window = Window.partitionBy("store_id").orderBy("transaction_date")#核心
daily_sales = daily_sales.withColumn("store_sales_T-1", lag("store_sales_T").over(window))
daily_sales = daily_sales.filter(col("store_sales_T-1").isNotNull())

daily_sales.show(5)
daily_sales.count()

+--------+----------------+-------------+---------------+
|store_id|transaction_date|store_sales_T|store_sales_T-1|
+--------+----------------+-------------+---------------+
|       1|      2023-07-02|      66456.0|        67301.5|
|       1|      2023-07-03|      67628.5|        66456.0|
|       1|      2023-07-04|      65243.5|        67628.5|
|       1|      2023-07-05|      69179.0|        65243.5|
|       1|      2023-07-06|      66620.5|        69179.0|
+--------+----------------+-------------+---------------+
only showing top 5 rows



7300

drink types

In [ ]:
transactions_join = transactions.select("transaction_id", "store_id")
transaction_items = transaction_items.join(transactions_join, on="transaction_id", how="left")

In [ ]:
menu = menu.select(col("item_id"), col("item_name").alias("drink_name"))
transaction_items = transaction_items.join(menu, on="item_id", how="left")

In [ ]:
drink_sales = transaction_items.groupBy("store_id", "transaction_date", "drink_name") \
    .agg(sum("subtotal").alias("item_sales"))

drink_types = ["Espresso", "Americano", "Latte", "Cappuccino", "Flat White", "Mocha", "Hot Chocolate", "Matcha Latte"]

drink_sales_new = drink_sales.groupBy("store_id", "transaction_date") \
    .pivot("drink_name", drink_types).sum("item_sales")
drink_sales_new = drink_sales_new.fillna(0)

drink_sales_new.show(5)
drink_sales_new.count()

+--------+----------------+--------+---------+------+----------+----------+------+-------------+------------+
|store_id|transaction_date|Espresso|Americano| Latte|Cappuccino|Flat White| Mocha|Hot Chocolate|Matcha Latte|
+--------+----------------+--------+---------+------+----------+----------+------+-------------+------------+
|       2|      2024-03-23|  5928.0|   6797.0|8216.0|    7952.0|    9558.0|9424.0|       9738.0|     10050.0|
|       4|      2024-08-29|  6126.0|   6566.0|7432.0|    7136.0|    8838.0|8920.5|       9234.0|      9580.0|
|       7|      2025-02-21|  5982.0|   7014.0|7848.0|    7904.0|    8982.0|9063.0|       9009.0|      9970.0|
|       2|      2025-04-10|  6546.0|   7014.0|8232.0|    7584.0|    8955.0|9956.0|       8388.0|      9370.0|
|       8|      2024-01-27|  5844.0|   7042.0|8600.0|    8008.0|    9081.0|9471.5|      10296.0|     10540.0|
+--------+----------------+--------+---------+------+----------+----------+------+-------------+------------+
only showi

7310

gender & average age

In [ ]:
users = users.withColumn("birthdate", to_date(col("birthdate")))
users = users.withColumn("registered_date", to_date(col("registered_at")))

transactions_users = transactions.join(users, on="user_id", how="left")

transactions_users = transactions_users.withColumn(
    "age_at_transaction",
    (datediff(col("transaction_date"), col("birthdate")) / 365).cast("double")
)

user_stats = transactions_users.groupBy("store_id", "transaction_date").agg(
    count(when(col("gender") == "male", True)).alias("num_male_T-1"),
    count(when(col("gender") == "female", True)).alias("num_female_T-1"),
    count(when(col("gender").isNull(), True)).alias("num_unknown_T-1"),
    round(avg("age_at_transaction")).alias("avg_age_T-1")
)

user_stats.show(5)
user_stats.count()

+--------+----------------+------------+--------------+---------------+-----------+
|store_id|transaction_date|num_male_T-1|num_female_T-1|num_unknown_T-1|avg_age_T-1|
+--------+----------------+------------+--------------+---------------+-----------+
|       1|      2023-09-14|          17|            13|           2035|       34.0|
|      10|      2025-05-27|         996|           982|             24|       39.0|
|       1|      2023-09-16|          12|             6|           1905|       36.0|
|       8|      2023-09-17|           5|             9|           1908|       40.0|
|       1|      2024-12-31|         949|           996|             93|       38.0|
+--------+----------------+------------+--------------+---------------+-----------+
only showing top 5 rows



7310

final table

In [ ]:
final = daily_sales.join(drink_sales_new, on=["store_id","transaction_date"], how="left")
final = final.join(user_stats, on=["store_id","transaction_date"], how="left")

final = final.fillna(0)

final = final.orderBy(col("store_id").asc(), col("transaction_date").asc())

final_output_path = 'file:/content/gdrive/MyDrive/CS4225/Assignment_2/final.csv'
final.write.csv(final_output_path, header=True, mode="overwrite")

print("Transactions:",final.count(), "rows")

Transactions: 7300 rows


## 2. Build ML Pipeline

reload dataset

In [ ]:
final = spark.read.csv(
    "file:/content/gdrive/MyDrive/CS4225/Assignment_2/final.csv",
    header=True,
    inferSchema=True
)

print("Rows:", final.count())
print("Columns:", final.columns)

final.show(5)

Rows: 7300
Columns: ['store_id', 'transaction_date', 'store_sales_T', 'store_sales_T-1', 'Espresso', 'Americano', 'Latte', 'Cappuccino', 'Flat White', 'Mocha', 'Hot Chocolate', 'Matcha Latte', 'num_male_T-1', 'num_female_T-1', 'num_unknown_T-1', 'avg_age_T-1']
+--------+----------------+-------------+---------------+--------+---------+------+----------+----------+-------+-------------+------------+------------+--------------+---------------+-----------+
|store_id|transaction_date|store_sales_T|store_sales_T-1|Espresso|Americano| Latte|Cappuccino|Flat White|  Mocha|Hot Chocolate|Matcha Latte|num_male_T-1|num_female_T-1|num_unknown_T-1|avg_age_T-1|
+--------+----------------+-------------+---------------+--------+---------+------+----------+----------+-------+-------------+------------+------------+--------------+---------------+-----------+
|       1|      2023-07-02|      66456.0|        67301.5|  5754.0|   7357.0|8112.0|    7912.0|    9522.0| 9633.0|       8316.0|      9850.0|        

train and test split

In [ ]:
train_data = final.filter((col("transaction_date") >= "2023-07-01") &
                           (col("transaction_date") <= "2024-12-31"))

test_data = final.filter((col("transaction_date") >= "2025-01-01") &
                          (col("transaction_date") <= "2025-06-30"))

print("Training rows:", train_data.count())
print("Testing rows:", test_data.count())

Training rows: 5490
Testing rows: 1810


pipeline

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline

feature_selected = [c for c, dtype in final.dtypes if dtype in ('double', 'int') and c not in ['store_sales_T']]
print("Selected features:", feature_selected)

assembler = VectorAssembler(inputCols=feature_selected, outputCol="features")

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="store_sales_T")

pipeline = Pipeline(stages=[assembler, scaler, rf])

model = pipeline.fit(train_data)

Selected features: ['store_id', 'store_sales_T-1', 'Espresso', 'Americano', 'Latte', 'Cappuccino', 'Flat White', 'Mocha', 'Hot Chocolate', 'Matcha Latte', 'num_male_T-1', 'num_female_T-1', 'num_unknown_T-1', 'avg_age_T-1']


## 3. Evaluate ML Pipeline

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

train_pred = model.transform(train_data)
test_pred = model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="store_sales_T", predictionCol="prediction", metricName="mae")

train_mae = evaluator.evaluate(train_pred)
test_mae = evaluator.evaluate(test_pred)

print("Training MAE:", train_mae)
print("Testing MAE:", test_mae)


Training MAE: 596.071688481766
Testing MAE: 624.0888361744389
